In [1]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from functions_pytorch import TripletGenerator,loss_func_generator,l2_distance
from pytorch_util import fine_tune_pretrainedmodels,fit,predict,trainable_parameter,set_requires_grad
from albumentations_helper import create_transform
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
import pickle
import numpy as np
import pretrainedmodels

In [2]:
## configs ##
color = True
#shapes = (224,224,1)
HalfBatch = 8
margin = -2
outDim = 500
distanceFun = l2_distance

In [3]:
if color:
    with open('/home/will/Desktop/kaggle/Whale/train_df_color.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train_color.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df_color.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val_color.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)
else:
    with open('/home/will/Desktop/kaggle/Whale/train_df.pkl', 'rb') as f:
        Ids_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_train.pkl', 'rb') as f:
        newWhale_train = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/val_df.pkl', 'rb') as f:
        Ids_val = pickle.load(f)
    with open('/home/will/Desktop/kaggle/Whale/new_whale_val.pkl', 'rb') as f:
        newWhale_val = pickle.load(f)

Set up dataloader

In [4]:
aug = Compose([RandomContrast(p=0.5),RandomBrightness(p=0.5),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=25,scale_limit=0.05,p=1),Cutout(p=0.5)])
transform = create_transform(aug)  

aug_test = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform_test = create_transform(aug_test)    

In [5]:
gen_train = TripletGenerator(Ids_train,newWhale_train,transform)
gen_val = TripletGenerator(Ids_val,newWhale_val,transform_test)        
#gen_train = FunctionWrapOverDataset(gen_train,numpy2torch)
#gen_val = FunctionWrapOverDataset(gen_val,numpy2torch)
train_dl= DataLoader(gen_train,HalfBatch,True,num_workers=3,drop_last=True)
valid_dl = DataLoader(gen_val,HalfBatch,False,num_workers=3,drop_last=True)

Set up training

In [10]:
model = pretrainedmodels.resnet50()
model = fine_tune_pretrainedmodels(model, outDim).to('cuda:0')

In [11]:
loss_func = loss_func_generator(HalfBatch,margin,distanceFun)

In [12]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [13]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-0.8073906029626727, val_loss:-1.3411529064178467
epoch:1, train_loss:-1.3800526036024094, val_loss:-1.466355562210083
epoch:2, train_loss:-1.4621889748811723, val_loss:-1.5512768030166626
epoch:3, train_loss:-1.5072017777919768, val_loss:-1.5741666555404663
epoch:4, train_loss:-1.5198405936717987, val_loss:-1.6213494539260864
Training completed in 156.37421441078186s


Fine-tune previous layers

In [18]:
set_requires_grad([model.layer4,model.layer3,model.layer2],True)

In [22]:
opt = Adam([
            {"params": model.layer4.parameters(), "lr": 1e-5},
            {"params": model.layer3.parameters(), "lr": 1e-6},
            {"params": model.layer2.parameters(), "lr": 5e-7},
            {"params": model.last_linear.parameters(), "lr": 2e-5},
            ])

In [23]:
model = fit(5, model, loss_func, opt, train_dl, valid_dl)

epoch:0, train_loss:-1.6213649644374848, val_loss:-1.6939446926116943
epoch:1, train_loss:-1.7082042113780975, val_loss:-1.7386196851730347
epoch:2, train_loss:-1.7758589567661285, val_loss:-1.7524770498275757
epoch:3, train_loss:-1.8027709209442138, val_loss:-1.7713443040847778
epoch:4, train_loss:-1.7992635990142822, val_loss:-1.752054214477539
Training completed in 572.0552089214325s
